In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from lxml import etree
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
check=[]
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv('salesdaily.csv')
df

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name
0,1/2/2014,0.00,3.670,3.40,32.40,7.0,0.0,0.0,2.00,2014,1,248,Thursday
1,1/3/2014,8.00,4.000,4.40,50.60,16.0,0.0,20.0,4.00,2014,1,276,Friday
2,1/4/2014,2.00,1.000,6.50,61.85,10.0,0.0,9.0,1.00,2014,1,276,Saturday
3,1/5/2014,4.00,3.000,7.00,41.10,8.0,0.0,3.0,0.00,2014,1,276,Sunday
4,1/6/2014,5.00,1.000,4.50,21.70,16.0,2.0,6.0,2.00,2014,1,276,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,10/4/2019,7.34,5.683,2.25,22.45,13.0,0.0,1.0,1.00,2019,10,276,Friday
2102,10/5/2019,3.84,5.010,6.00,25.40,7.0,0.0,0.0,0.33,2019,10,276,Saturday
2103,10/6/2019,4.00,11.690,2.00,34.60,6.0,0.0,5.0,4.20,2019,10,276,Sunday
2104,10/7/2019,7.34,4.507,3.00,50.80,6.0,0.0,10.0,1.00,2019,10,276,Monday


In [3]:
Datelist=df.loc[:,"datum"]
Datelist

0        1/2/2014
1        1/3/2014
2        1/4/2014
3        1/5/2014
4        1/6/2014
          ...    
2101    10/4/2019
2102    10/5/2019
2103    10/6/2019
2104    10/7/2019
2105    10/8/2019
Name: datum, Length: 2106, dtype: object

In [4]:
Datelist.values.tolist()
Datelist

0        1/2/2014
1        1/3/2014
2        1/4/2014
3        1/5/2014
4        1/6/2014
          ...    
2101    10/4/2019
2102    10/5/2019
2103    10/6/2019
2104    10/7/2019
2105    10/8/2019
Name: datum, Length: 2106, dtype: object

In [5]:
check= pd.read_csv('check.csv' ,index_col =False,header=None ,squeeze = True)
check.values.tolist()
alpha=[]
for i in range(0,len(check)):
    alpha.append(datetime.strptime(str(check[i]), '%d/%m/%Y').date())

In [6]:
import time as t
def wait_for_class_to_be_available(browser, total_wait=30):

    wait = WebDriverWait(browser, 500)    
    elem =wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "mat-table.cdk-table.mat-sort.ng-star-inserted")))
    return(browser.page_source)
def indexing(html1,i):
    df1 = pd.DataFrame(columns=['date','Time','Temperature','Dew Point','Humidity','Wind','Wind Speed','Wind Gust','Pressure','Precip.','Condition'])
    soup1 = BeautifulSoup(html1,"lxml")
    table= soup1.find_all('table')
    table = soup1.find('table', class_=('mat-table cdk-table mat-sort ng-star-inserted'))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        if(columns != []):
            time=columns[0].text.strip()
            temperature=columns[1].text.strip()
            dew_Point=columns[2].text.strip()
            humidity=columns[3].text.strip()
            wind=columns[4].text.strip()
            wind_Speed=columns[5].text.strip()
            wind_Gust=columns[6].text.strip()
            pressure=columns[7].text.strip()
            precip=columns[8].text.strip()
            condition=columns[9].text.strip()
            df1 = df1.append({'date':i,'Time':time,'Temperature':temperature,'Dew Point':dew_Point,'Humidity':humidity,'Wind':wind,'Wind Speed':wind_Speed,'Wind Gust':wind_Gust,'Pressure':pressure,'Precip.':precip,'Condition':condition}, ignore_index=True)
    return df1
def RemoveAssci(df1,i):
    df1['Temperature']=df1['Temperature'].str.encode('ascii', 'ignore').str.decode('ascii')
    df1['Dew Point']=df1['Dew Point'].str.encode('ascii', 'ignore').str.decode('ascii')
    df1['Humidity']=df1['Humidity'].str.encode('ascii', 'ignore').str.decode('ascii')
    df1['Wind Speed']=df1['Wind Speed'].str.encode('ascii', 'ignore').str.decode('ascii')
    df1['Pressure']=df1['Pressure'].str.encode('ascii', 'ignore').str.decode('ascii')
    df1['Precip.']=df1['Precip.'].str.encode('ascii', 'ignore').str.decode('ascii')
    df1['Wind Gust']=df1['Wind Gust'].str.encode('ascii', 'ignore').str.decode('ascii')
    df1['Temperature']=df1['Temperature'].str.replace('F',' ')
    df1['Dew Point']=df1['Dew Point'].str.replace('F',' ')
    df1['Humidity']=df1['Humidity'].str.replace('%',' ')
    df1['Wind Speed']=df1['Wind Speed'].str.replace('mph',' ')
    df1['Pressure']=df1['Pressure'].str.replace('in',' ')
    df1['Precip.']=df1['Precip.'].str.replace('in',' ')
    df1['Wind Gust']=df1['Wind Gust'].str.replace('mph',' ')
    alpha.append(str(i))
    df1.to_csv('main.csv', mode='a', index=False, header=False)
    
    return (i)



driver = webdriver.Chrome('C:/chromedriver/chromedriver')
for index in range(0,len(Datelist)):
    datetime_object = datetime.strptime(str(Datelist[index]), '%m/%d/%Y').date()
#     print((alpha.count(datetime_object)),datetime_object)
#     print(datetime_object in alpha)
    if not (datetime_object in alpha):
        
        driver.get("https://www.wunderground.com/history/daily/pk/karachi/OPKC/date/"+str(datetime_object))
        html1=wait_for_class_to_be_available(driver)
        df2=indexing(html1,datetime_object)
        print(RemoveAssci(df2,datetime_object))
    else:
        index=index+1
        
        
    


In [7]:
lst=alpha
chk = pd.DataFrame(lst)
chk.to_csv('check.csv', index=False)




In [5]:
main = pd.read_csv('main.csv')
main

,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,02/01/2014,11:55 pm,55,23,28,NNE,14,0,30.10,0,Fair
1,02/01/2014,12:25 am,55,23,28,NNE,12,0,30.10,0,Fair
2,02/01/2014,12:55 am,54,21,28,NNE,7,0,30.10,0,Fair
3,02/01/2014,1:25 am,54,23,30,NNE,9,0,30.10,0,Fair
4,02/01/2014,2:25 am,50,23,35,NNE,7,0,30.10,0,Fair
...,...,...,...,...,...,...,...,...,...,...,...
89977,08/10/2019,9:55 pm,82,72,70,WSW,6,0,29.81,0,Fair
89978,08/10/2019,10:25 pm,82,72,70,W,5,0,29.81,0,Fair
89979,08/10/2019,10:59 pm,81,72,74,W,5,0,29.81,0,Fair
89980,08/10/2019,11:25 pm,81,72,74,W,7,0,29.81,0,Fair


In [6]:
saleshourly = pd.read_csv('saleshourly.csv')
saleshourly

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name
0,01/02/2014 8:00,0.00,0.67,0.4,2.0,0.0,0.0,0.0,1.0,2014,1,8,Thursday
1,01/02/2014 9:00,0.00,0.00,1.0,0.0,2.0,0.0,0.0,0.0,2014,1,9,Thursday
2,01/02/2014 10:00,0.00,0.00,0.0,3.0,2.0,0.0,0.0,0.0,2014,1,10,Thursday
3,01/02/2014 11:00,0.00,0.00,0.0,2.0,1.0,0.0,0.0,0.0,2014,1,11,Thursday
4,01/02/2014 12:00,0.00,2.00,0.0,5.0,2.0,0.0,0.0,0.0,2014,1,12,Thursday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50527,10/08/2019 15:00,0.00,0.40,0.0,11.3,1.0,0.0,0.0,0.0,2019,10,15,Tuesday
50528,10/08/2019 16:00,0.33,0.00,0.0,1.0,2.0,0.0,0.0,0.0,2019,10,16,Tuesday
50529,10/08/2019 17:00,0.00,0.00,0.0,1.0,2.0,0.0,1.0,0.0,2019,10,17,Tuesday
50530,10/08/2019 18:00,0.00,0.00,0.0,9.0,0.0,0.0,0.0,0.0,2019,10,18,Tuesday


In [7]:
tm=main.loc[:,"Time"]
newclm=[]
for i in tm:
    ts=pd.Timestamp(str(i))
    newclm.append(str(ts.round(freq='h').time()))
    
main['Hour']=newclm
main

,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Hour
0,02/01/2014,11:55 pm,55,23,28,NNE,14,0,30.10,0,Fair,00:00:00
1,02/01/2014,12:25 am,55,23,28,NNE,12,0,30.10,0,Fair,00:00:00
2,02/01/2014,12:55 am,54,21,28,NNE,7,0,30.10,0,Fair,01:00:00
3,02/01/2014,1:25 am,54,23,30,NNE,9,0,30.10,0,Fair,01:00:00
4,02/01/2014,2:25 am,50,23,35,NNE,7,0,30.10,0,Fair,02:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
89977,08/10/2019,9:55 pm,82,72,70,WSW,6,0,29.81,0,Fair,22:00:00
89978,08/10/2019,10:25 pm,82,72,70,W,5,0,29.81,0,Fair,22:00:00
89979,08/10/2019,10:59 pm,81,72,74,W,5,0,29.81,0,Fair,23:00:00
89980,08/10/2019,11:25 pm,81,72,74,W,7,0,29.81,0,Fair,23:00:00


In [8]:
sh=saleshourly.loc[:,'datum']
sh.values.tolist()
sep_date=[]
sep_day=[]
for i in range(0,len(sh)):
    z=sh[i].split("/")[1]
    sep_day.append(z)
    t_sa=datetime.strptime(str(sh[i]), '%m/%d/%Y %H:%M')
    sep_date.append(t_sa.date())
    
    
saleshourly['datum']=sep_date 
saleshourly['Day'] =sep_day
saleshourly

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name,Day
0,2014-01-02,0.00,0.67,0.4,2.0,0.0,0.0,0.0,1.0,2014,1,8,Thursday,02
1,2014-01-02,0.00,0.00,1.0,0.0,2.0,0.0,0.0,0.0,2014,1,9,Thursday,02
2,2014-01-02,0.00,0.00,0.0,3.0,2.0,0.0,0.0,0.0,2014,1,10,Thursday,02
3,2014-01-02,0.00,0.00,0.0,2.0,1.0,0.0,0.0,0.0,2014,1,11,Thursday,02
4,2014-01-02,0.00,2.00,0.0,5.0,2.0,0.0,0.0,0.0,2014,1,12,Thursday,02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50527,2019-10-08,0.00,0.40,0.0,11.3,1.0,0.0,0.0,0.0,2019,10,15,Tuesday,08
50528,2019-10-08,0.33,0.00,0.0,1.0,2.0,0.0,0.0,0.0,2019,10,16,Tuesday,08
50529,2019-10-08,0.00,0.00,0.0,1.0,2.0,0.0,1.0,0.0,2019,10,17,Tuesday,08
50530,2019-10-08,0.00,0.00,0.0,9.0,0.0,0.0,0.0,0.0,2019,10,18,Tuesday,08


In [9]:
main['Hour'] = main['Hour'].str.split(":", expand = True)[0]
main[['Day', 'Month', 'Year']] = main['Date'].str.split("/", expand = True)
main=main.drop_duplicates(subset=['Day','Month','Year' ,'Hour'],  keep='first', inplace=False, ignore_index=False)
main 
    

,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Hour,Day,Month,Year
0,02/01/2014,11:55 pm,55,23,28,NNE,14,0,30.10,0,Fair,00,02,01,2014
2,02/01/2014,12:55 am,54,21,28,NNE,7,0,30.10,0,Fair,01,02,01,2014
4,02/01/2014,2:25 am,50,23,35,NNE,7,0,30.10,0,Fair,02,02,01,2014
5,02/01/2014,2:55 am,50,23,35,NNE,5,0,30.07,0,Fair,03,02,01,2014
7,02/01/2014,4:25 am,50,21,32,NNE,6,0,30.07,0,Fair,04,02,01,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89971,08/10/2019,7:00 pm,86,68,55,SW,9,0,29.78,0,Fair,19,08,10,2019
89972,08/10/2019,7:30 pm,84,70,62,SW,9,0,29.81,0,Fair,20,08,10,2019
89975,08/10/2019,9:00 pm,82,70,66,SW,12,0,29.81,0,Fair,21,08,10,2019
89977,08/10/2019,9:55 pm,82,72,70,WSW,6,0,29.81,0,Fair,22,08,10,2019


In [10]:
import numpy as np
main['Year']=main['Year'].astype(int)
main['Day']=main['Day'].astype(int)
main['Hour']=main['Hour'].astype(int)
main['Month']=main['Month'].astype(int)
saleshourly['Year']=saleshourly['Year'].astype(int)
saleshourly['Day']=saleshourly['Day'].astype(int)
saleshourly['Hour']=saleshourly['Hour'].astype(int)
saleshourly['Month']=saleshourly['Month'].astype(int)
output1 = pd.merge(saleshourly, main,  
                   on=['Day','Month','Year' ,'Hour'],  
                   how='left') 
output1

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,...,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2014-01-02,0.00,0.67,0.4,2.0,0.0,0.0,0.0,1.0,2014,...,7:55 am,48.0,21.0,34.0,N,6.0,0.0,30.10,0.0,Fair
1,2014-01-02,0.00,0.00,1.0,0.0,2.0,0.0,0.0,0.0,2014,...,8:55 am,54.0,23.0,30.0,NNE,8.0,0.0,30.13,0.0,Fair
2,2014-01-02,0.00,0.00,0.0,3.0,2.0,0.0,0.0,0.0,2014,...,10:25 am,61.0,23.0,23.0,NE,14.0,0.0,30.16,0.0,Fair
3,2014-01-02,0.00,0.00,0.0,2.0,1.0,0.0,0.0,0.0,2014,...,10:55 am,64.0,19.0,18.0,NE,15.0,0.0,30.16,0.0,Fair
4,2014-01-02,0.00,2.00,0.0,5.0,2.0,0.0,0.0,0.0,2014,...,11:55 am,66.0,16.0,14.0,NE,13.0,0.0,30.13,0.0,Fair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50527,2019-10-08,0.00,0.40,0.0,11.3,1.0,0.0,0.0,0.0,2019,...,3:00 pm,91.0,68.0,46.0,SW,10.0,0.0,29.78,0.0,Fair
50528,2019-10-08,0.33,0.00,0.0,1.0,2.0,0.0,0.0,0.0,2019,...,3:30 pm,91.0,68.0,46.0,WSW,9.0,0.0,29.78,0.0,Fair
50529,2019-10-08,0.00,0.00,0.0,1.0,2.0,0.0,1.0,0.0,2019,...,5:00 pm,91.0,66.0,43.0,SSW,16.0,0.0,29.78,0.0,Fair
50530,2019-10-08,0.00,0.00,0.0,9.0,0.0,0.0,0.0,0.0,2019,...,5:30 pm,90.0,66.0,46.0,SSW,14.0,0.0,29.78,0.0,Fair


In [11]:
output1['M01AB']=output1['M01AB'].round()
output1['M01AE']=output1['M01AE'].round()
output1['N02BA']=output1['N02BA'].round()
output1['N02BE']=output1['N02BE'].round()
output1['N05B']=output1['N05B'].round()
output1['N05C']=output1['N05C'].round()
output1['R03']=output1['R03'].round()
output1['R06']=output1['R06'].round()
output1['M01AB']=output1['M01AB'].astype(int)
output1['M01AE']=output1['M01AE'].astype(int)
output1['N02BA']=output1['N02BA'].astype(int)
output1['N02BE']=output1['N02BE'].astype(int)
output1['N05B']=output1['N05B'].astype(int)
output1['N05C']=output1['N05C'].astype(int)
output1['R03']=output1['R03'].astype(int)
output1['R06']=output1['R06'].astype(int)
output1

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,...,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2014-01-02,0,1,0,2,0,0,0,1,2014,...,7:55 am,48.0,21.0,34.0,N,6.0,0.0,30.10,0.0,Fair
1,2014-01-02,0,0,1,0,2,0,0,0,2014,...,8:55 am,54.0,23.0,30.0,NNE,8.0,0.0,30.13,0.0,Fair
2,2014-01-02,0,0,0,3,2,0,0,0,2014,...,10:25 am,61.0,23.0,23.0,NE,14.0,0.0,30.16,0.0,Fair
3,2014-01-02,0,0,0,2,1,0,0,0,2014,...,10:55 am,64.0,19.0,18.0,NE,15.0,0.0,30.16,0.0,Fair
4,2014-01-02,0,2,0,5,2,0,0,0,2014,...,11:55 am,66.0,16.0,14.0,NE,13.0,0.0,30.13,0.0,Fair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50527,2019-10-08,0,0,0,11,1,0,0,0,2019,...,3:00 pm,91.0,68.0,46.0,SW,10.0,0.0,29.78,0.0,Fair
50528,2019-10-08,0,0,0,1,2,0,0,0,2019,...,3:30 pm,91.0,68.0,46.0,WSW,9.0,0.0,29.78,0.0,Fair
50529,2019-10-08,0,0,0,1,2,0,1,0,2019,...,5:00 pm,91.0,66.0,43.0,SSW,16.0,0.0,29.78,0.0,Fair
50530,2019-10-08,0,0,0,9,0,0,0,0,2019,...,5:30 pm,90.0,66.0,46.0,SSW,14.0,0.0,29.78,0.0,Fair


In [12]:
output1.head(200)

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,...,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition
0,2014-01-02,0,1,0,2,0,0,0,1,2014,...,7:55 am,48.0,21.0,34.0,N,6.0,0.0,30.10,0.0,Fair
1,2014-01-02,0,0,1,0,2,0,0,0,2014,...,8:55 am,54.0,23.0,30.0,NNE,8.0,0.0,30.13,0.0,Fair
2,2014-01-02,0,0,0,3,2,0,0,0,2014,...,10:25 am,61.0,23.0,23.0,NE,14.0,0.0,30.16,0.0,Fair
3,2014-01-02,0,0,0,2,1,0,0,0,2014,...,10:55 am,64.0,19.0,18.0,NE,15.0,0.0,30.16,0.0,Fair
4,2014-01-02,0,2,0,5,2,0,0,0,2014,...,11:55 am,66.0,16.0,14.0,NE,13.0,0.0,30.13,0.0,Fair
5,2014-01-02,0,1,2,20,0,0,0,1,2014,...,12:55 pm,68.0,12.0,12.0,NE,25.0,0.0,30.10,0.0,Fair / Windy
6,2014-01-02,0,0,0,0,0,0,0,0,2014,...,1:55 pm,70.0,12.0,11.0,ENE,26.0,0.0,30.04,0.0,Fair / Windy
7,2014-01-02,0,0,0,0,0,0,0,0,2014,...,2:55 pm,70.0,14.0,12.0,NNE,21.0,0.0,30.01,0.0,Fair / Windy
8,2014-01-02,0,0,0,0,0,0,0,0,2014,...,3:55 pm,70.0,14.0,12.0,NE,25.0,0.0,30.01,0.0,Fair / Windy
9,2014-01-02,0,0,0,0,0,0,0,0,2014,...,4:59 pm,70.0,10.0,10.0,NNE,17.0,0.0,30.01,0.0,Fair


In [13]:
import pandas as pd

# Assuming output1 is a DataFrame containing the 'Condition' column
lste = output1['Condition'].unique()

# Create a dictionary to map unique values to integer codes
code_map = {val: i+1 for i, val in enumerate(lste)}

# Use the map() method to assign integer codes to each value in the 'Condition' column
output1['IntCondition'] = output1['Condition'].map(code_map)

# Create a DataFrame from the unique values with their assigned integer codes
fair = pd.DataFrame({'Condition': list(code_map.keys()), 'IntCondition': list(code_map.values())})

# Display the resulting DataFrame
print(fair)


                  Condition  IntCondition
0                      Fair             1
1              Fair / Windy             2
2                       NaN             3
3             Partly Cloudy             4
4             Mostly Cloudy             5
5     Partly Cloudy / Windy             6
6     Mostly Cloudy / Windy             7
7                      Haze             8
8                     Smoke             9
9           Widespread Dust            10
10  Thunder in the Vicinity            11
11                  T-Storm            12
12     Blowing Dust / Windy            13
13               Light Rain            14
14             Haze / Windy            15
15                  Drizzle            16
16             Blowing Dust            17
17          Drizzle / Windy            18
18           Cloudy / Windy            19
19                   Cloudy            20
20                     Rain            21
21  Widespread Dust / Windy            22
22                      Fog       

In [14]:
output1.dropna(subset=['Date','Time','Temperature','Dew Point','Humidity','Wind','Wind Speed','Wind Gust','Pressure','Precip.','Condition'], inplace=True)
pd.set_option('display.max_columns', None)
output1.head(200)

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name,Day,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,IntCondition
0,2014-01-02,0,1,0,2,0,0,0,1,2014,1,8,Thursday,2,02/01/2014,7:55 am,48.0,21.0,34.0,N,6.0,0.0,30.10,0.0,Fair,1
1,2014-01-02,0,0,1,0,2,0,0,0,2014,1,9,Thursday,2,02/01/2014,8:55 am,54.0,23.0,30.0,NNE,8.0,0.0,30.13,0.0,Fair,1
2,2014-01-02,0,0,0,3,2,0,0,0,2014,1,10,Thursday,2,02/01/2014,10:25 am,61.0,23.0,23.0,NE,14.0,0.0,30.16,0.0,Fair,1
3,2014-01-02,0,0,0,2,1,0,0,0,2014,1,11,Thursday,2,02/01/2014,10:55 am,64.0,19.0,18.0,NE,15.0,0.0,30.16,0.0,Fair,1
4,2014-01-02,0,2,0,5,2,0,0,0,2014,1,12,Thursday,2,02/01/2014,11:55 am,66.0,16.0,14.0,NE,13.0,0.0,30.13,0.0,Fair,1
5,2014-01-02,0,1,2,20,0,0,0,1,2014,1,13,Thursday,2,02/01/2014,12:55 pm,68.0,12.0,12.0,NE,25.0,0.0,30.10,0.0,Fair / Windy,2
6,2014-01-02,0,0,0,0,0,0,0,0,2014,1,14,Thursday,2,02/01/2014,1:55 pm,70.0,12.0,11.0,ENE,26.0,0.0,30.04,0.0,Fair / Windy,2
7,2014-01-02,0,0,0,0,0,0,0,0,2014,1,15,Thursday,2,02/01/2014,2:55 pm,70.0,14.0,12.0,NNE,21.0,0.0,30.01,0.0,Fair / Windy,2
8,2014-01-02,0,0,0,0,0,0,0,0,2014,1,16,Thursday,2,02/01/2014,3:55 pm,70.0,14.0,12.0,NE,25.0,0.0,30.01,0.0,Fair / Windy,2
9,2014-01-02,0,0,0,0,0,0,0,0,2014,1,17,Thursday,2,02/01/2014,4:59 pm,70.0,10.0,10.0,NNE,17.0,0.0,30.01,0.0,Fair,1


In [15]:
import pandas as pd

# Assuming your DataFrame is stored in a variable named 'df' and the column with the wind directions is named 'WindDir'
wind_directions = {
    'N': 0,
    'NNE': 23,
    'NE': 45,
    'ENE': 68,
    'E': 90,
    'ESE': 113,
    'SE': 135,
    'SSE': 158,
    'S': 180,
    'SSW': 203,
    'SW': 225,
    'WSW': 248,
    'W': 270,
    'WNW': 293,
    'NW': 315,
    'NNW': 338,
    'CALM': 0,
    'VAR': 0
}

# Replace the wind direction values with their corresponding integers
output1['Wind'] = output1['Wind'].replace(wind_directions)

# Display the resulting DataFrame
print(output1)


            datum  M01AB  M01AE  N02BA  N02BE  N05B  N05C  R03  R06  Year  \
0      2014-01-02      0      1      0      2     0     0    0    1  2014   
1      2014-01-02      0      0      1      0     2     0    0    0  2014   
2      2014-01-02      0      0      0      3     2     0    0    0  2014   
3      2014-01-02      0      0      0      2     1     0    0    0  2014   
4      2014-01-02      0      2      0      5     2     0    0    0  2014   
...           ...    ...    ...    ...    ...   ...   ...  ...  ...   ...   
50527  2019-10-08      0      0      0     11     1     0    0    0  2019   
50528  2019-10-08      0      0      0      1     2     0    0    0  2019   
50529  2019-10-08      0      0      0      1     2     0    1    0  2019   
50530  2019-10-08      0      0      0      9     0     0    0    0  2019   
50531  2019-10-08      0      0      0      8     1     0    1    0  2019   

       Month  Hour Weekday Name  Day        Date      Time  Temperature  \


In [16]:
weekly=output1
monthly=output1
yearly=output1
weekly.head(500)

,datum,M01AB,M01AE,N02BA,N02BE,N05B,N05C,R03,R06,Year,Month,Hour,Weekday Name,Day,Date,Time,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,IntCondition
0,2014-01-02,0,1,0,2,0,0,0,1,2014,1,8,Thursday,2,02/01/2014,7:55 am,48.0,21.0,34.0,0,6.0,0.0,30.10,0.0,Fair,1
1,2014-01-02,0,0,1,0,2,0,0,0,2014,1,9,Thursday,2,02/01/2014,8:55 am,54.0,23.0,30.0,23,8.0,0.0,30.13,0.0,Fair,1
2,2014-01-02,0,0,0,3,2,0,0,0,2014,1,10,Thursday,2,02/01/2014,10:25 am,61.0,23.0,23.0,45,14.0,0.0,30.16,0.0,Fair,1
3,2014-01-02,0,0,0,2,1,0,0,0,2014,1,11,Thursday,2,02/01/2014,10:55 am,64.0,19.0,18.0,45,15.0,0.0,30.16,0.0,Fair,1
4,2014-01-02,0,2,0,5,2,0,0,0,2014,1,12,Thursday,2,02/01/2014,11:55 am,66.0,16.0,14.0,45,13.0,0.0,30.13,0.0,Fair,1
5,2014-01-02,0,1,2,20,0,0,0,1,2014,1,13,Thursday,2,02/01/2014,12:55 pm,68.0,12.0,12.0,45,25.0,0.0,30.10,0.0,Fair / Windy,2
6,2014-01-02,0,0,0,0,0,0,0,0,2014,1,14,Thursday,2,02/01/2014,1:55 pm,70.0,12.0,11.0,68,26.0,0.0,30.04,0.0,Fair / Windy,2
7,2014-01-02,0,0,0,0,0,0,0,0,2014,1,15,Thursday,2,02/01/2014,2:55 pm,70.0,14.0,12.0,23,21.0,0.0,30.01,0.0,Fair / Windy,2
8,2014-01-02,0,0,0,0,0,0,0,0,2014,1,16,Thursday,2,02/01/2014,3:55 pm,70.0,14.0,12.0,45,25.0,0.0,30.01,0.0,Fair / Windy,2
9,2014-01-02,0,0,0,0,0,0,0,0,2014,1,17,Thursday,2,02/01/2014,4:59 pm,70.0,10.0,10.0,23,17.0,0.0,30.01,0.0,Fair,1


In [17]:
# weekly.drop(['Humidity','Wind','Wind Speed','Wind Gust','Pressure','Precip.','Condition','Condition_int','Time_Stamp','Date','Year','Month','Hour','Weekday Name','Day','Time','Temperature','Dew Point'], axis=1, inplace=True)
# weekly.head(20)
weekly.drop(['Date','Weekday Name','Time','Wind Gust','Precip.','Condition'], axis=1, inplace=True)
weekly.to_csv('CleanDataset.csv')

In [27]:
weekly["datum"] = pd.to_datetime(weekly["datum"])
weekly_summary =weekly.resample('w' ,on='datum').agg({ 'M01AB' :'sum', 'M01AE' : 'sum', 'N02BA' :'sum', 'N02BE' : 'sum', 'N05B' :'sum', 'N05C' : 'sum', 'R03' :'sum','R06' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean','Wind Gust' :'mean', 'Pressure' :'mean', 'Precip.' :'mean',  'Condition_int' : 'mean'})
weekly_summary.to_csv('Weekly.csv')

In [28]:
monthly["datum"] = pd.to_datetime(weekly["datum"])
Monthly_summary =monthly.resample('M' ,on='datum').agg({ 'M01AB' :'sum', 'M01AE' : 'sum', 'N02BA' :'sum', 'N02BE' : 'sum', 'N05B' :'sum', 'N05C' : 'sum', 'R03' :'sum','R06' :'sum', 'Temperature' :'mean', 'Dew Point' :'mean', 'Humidity' :'mean',  'Wind Speed' :'mean','Wind Gust' :'mean', 'Pressure' :'mean', 'Precip.' :'mean',  'Condition_int' : 'mean'})
Monthly_summary.to_csv('Monthly.csv')